In [ ]:
# !pip install pandas
# !pip install numpy

In [1]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None # Fuck warnings

# Загрузка данных

In [2]:
import json


with open(r'snli_1.0_train.jsonl', 'r') as json_file:
    json_list = list(json_file)

result = []

num = -1 # Сколько данных забрать (-1 чтобы забрать всё)

if num < 0:
    num = float('inf')

num = min(len(json_list), num)

for j in range(num):
    result.append(json.loads(json_list[j]))

In [3]:
df_train = pd.json_normalize(result)

In [4]:
df_train = df_train[['gold_label', 'sentence1', 'sentence2', 'sentence1_binary_parse', 'sentence2_binary_parse']]

In [5]:
df_train.columns

Index(['gold_label', 'sentence1', 'sentence2', 'sentence1_binary_parse',
       'sentence2_binary_parse'],
      dtype='object')

## Очистка от необозначенных классов

In [6]:
omission_mask = df_train['gold_label']== '-'
df_train = df_train.loc[~omission_mask]
df_train = df_train.reset_index(drop=True)


In [7]:
lenght = len(df_train)

## Очистка от бесконечностей в смысловых расстояниях

In [8]:
checker = pd.DataFrame()

dis = np.loadtxt('D:\Monty_Python_lmao\Fumo parsing\distances.txt')
dis = dis[:lenght]
checker[0]= dis

infinity_mask = checker[0]>999
indexs = df_train.loc[infinity_mask].index

df_train = df_train.drop(indexs, axis = 0)
df_train = df_train.reset_index(drop=True)
df_train

,gold_label,sentence1,sentence2,sentence1_binary_parse,sentence2_binary_parse
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ..."
3,neutral,Children smiling and waving at camera,They are smiling at their parents,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...
4,entailment,Children smiling and waving at camera,There are children present,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) )
...,...,...,...,...,...
549276,contradiction,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',( ( ( ( Four dirty ) and ) ( barefooted childr...,( ( four kids ) ( ( won awards ) ( ( ( for ` )...
549277,neutral,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",( ( ( ( Four dirty ) and ) ( barefooted childr...,( ( ( ( ( ( four ( homeless children ) ) ( had...
549278,neutral,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,( ( ( A man ) ( in ( a bodysuit ) ) ) ( ( is (...
549279,contradiction,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,( ( ( A man ) ( in ( a ( business suit ) ) ) )...


In [9]:
checker = checker.drop(indexs, axis = 0)
checker =checker.reset_index(drop=True)
checker

,0
0,0.792598
1,1.064395
2,0.796780
3,0.764126
4,1.007054
...,...
549276,0.861570
549277,0.694256
549278,0.669675
549279,1.108610


# Предобработка данных

## Оставляем только основу от слов

In [10]:
import nltk.stem


my_stem = nltk.stem.SnowballStemmer('english')

def stemSentence(sentence):
    new_sentence = ''

    for word in sentence.split():
        new_sentence += my_stem.stem(word) + ' '
    
    return new_sentence[:-1]

## Убираем символы

In [11]:
import string

def deleteSymbs(sentence):
    res = ''

    for char in sentence:
        if char not in string.punctuation or char in ['(', ')']:
            res += char.lower()
    
    return res

## Убираем не несущие смысла слова 

In [12]:
from nltk.corpus import stopwords

def deleteStopWords(sentence):
    res = ''
    
    for word in sentence.split():
        if word not in stopwords.words('english'):
            res += word + ' '
    
    return res[:-1]

## Считаем нет

In [13]:
def isNeg(sentence):
  counter = 0
  for word in sentence.split():
    if word in ['not', 'no', 'neither', 'none', 'nowhere', 'never', 'nothing']:
      counter += 1
    elif 'n`t' in word:
      counter += 1
  return (counter + 1) // 2

## Сколько одинаковых слов

In [14]:
def countSameWords(sentence1, sentence2):
  counter = 0
  for word in sentence1.split():
    if my_stem.stem(word) in stemSentence(sentence2).split():
      counter += 1
  return counter

## Подсчёт слов в предложении

In [15]:
def countWords(sentence):
  return len(sentence.split())

## Работаем со скобками

In [16]:
def getToCloseBracketPart(num_sym, sentence):
    while num_sym < len(sentence) and sentence[num_sym] != '(':
        num_sym += 1

    if num_sym == len(sentence):
        return 'Not a bracket, you fucker'

    sentence = sentence[num_sym + 1:]
    bracket_counter = 1
    for i, char in enumerate(sentence):
        if char == '(':
            bracket_counter += 1
        elif char == ')':
            bracket_counter -= 1
            if bracket_counter == 0:
                sentence = sentence[:i]
    return sentence


def delSpaces(sentence):
    res = ''
    i = 0
    while i < len(sentence):
        res += sentence[i]
        if sentence[i] == '(':
            i += 1
        elif sentence[i] == ')':
            res = res[:-2] + ')'
        i += 1
    return res


def unpackSentence(sentence):
    sentence = sentence.strip()
    this_part = ''
    parts = []

    if sentence[0] != '(':
        if '(' not in sentence:
            return [sentence]
        else:
            sentence = '(' + sentence[:sentence.find('(')] + ')' + sentence[sentence.find('('):]

    bracket_counter = 0
    for i, char in enumerate(sentence):
        if char == "(":
            bracket_counter += 1
            this_part += char
        elif char == ")":
            this_part += char
            bracket_counter -= 1
            if bracket_counter == 0:
                parts.append(this_part.strip())
                this_part = ''
        else:
            this_part += char

    if this_part != '':
      parts.append(f'({this_part.strip()})')
    new_parts = []
    for part in parts:
        if part != '':
            new_parts.append(part[1:-1])
    return new_parts


def indexMaxWords(array):
    index = 0
    maxis = 0
    for i, el in enumerate(array):
        if len(el.split()) > maxis:
            index = i
            maxis = len(el.split())
    return index


def unifySentences(sentence1, sentence2):
    smaller = unpackSentence(sentence1)
    bigger = unpackSentence(sentence2)

    if len(bigger) < len(smaller):
        bigger, smaller = smaller, bigger

    while len(smaller) != len(bigger) or len(smaller) < 3:
        index = indexMaxWords(smaller)
        new_parts = unpackSentence(smaller[index])
        if new_parts[0] == smaller[index]:
          new_parts.append(smaller[index])
        del smaller[index]
        smaller.extend(new_parts)

        if len(bigger) < len(smaller):
            bigger, smaller = smaller, bigger
    return bigger, smaller


def AllPossiblitieses():
    res = []
    for i in range(3):
        vec = [i, -1, -1]
        for j in set(range(3)) - set([i]):
            c = list(set(range(3)) - set([i]) - set([j]))[0]
            vec[1] = j
            vec[2] = c
            res.append(tuple(vec))
    return res


def getMeaningVecs(sentence1, sentence2):
  sentence1, sentence2 = unifySentences(sentence1, sentence2)
  sentence1 = list(map(getSentenceVecSum, sentence1))
  sentence2 = list(map(getSentenceVecSum, sentence2))

  sums = []
  possibilities = AllPossiblitieses()
  for pos in possibilities:
    sum = 0
    for i in range(3):
      take = sentence1[i] - sentence2[pos[i]]
      take = np.array(take)
      sum += np.sum(sum)
    sums.append(sum)
  
  sums = np.array(sums)
  best_pos = possibilities[np.argmin(sums)]

  vecs = []
  for i in range(3):
    vecs.append(sentence1[i] - sentence2[best_pos[i]])
  
  return vecs

## Agregate function

In [17]:
def fixSentence(sentence, funcs):   # funcs должно включать вышеописанные функции
    for func in funcs:
        sentence = eval(func)(sentence)
    
    return sentence

# Word2Vek и Doc2Vec

In [18]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api


# corpus = api.load('text8')  # download the corpus and return it opened as an iterable
# model = Word2Vec(corpus)    # train a model from the corpus
# model.most_similar("car")

In [19]:
# model = api.load('glove-twitter-200') # 758MB
model = api.load('glove-wiki-gigaword-100') # 128MB - гораздо адекватнее

In [20]:
for i in [1, 2]:
    df_train[f'Fixed_sentence{i}'] = df_train[f'sentence{i}'].apply(lambda sentence: fixSentence(sentence, ['deleteSymbs']))

In [21]:
for i in [1, 2]:
    df_train[f'Fixed_bin_sentence{i}'] = df_train[f'sentence{i}_binary_parse'].apply(lambda sentence: fixSentence(sentence, ['deleteSymbs', 'delSpaces']))

In [22]:
model.most_similar('horse') # Проверка работы скачанной модели word2vec

[('horses', 0.8550404906272888),
 ('riding', 0.7131650447845459),
 ('dog', 0.7109653353691101),
 ('racing', 0.6629325151443481),
 ('bike', 0.6614626049995422),
 ('thoroughbred', 0.6600428819656372),
 ('bull', 0.6456228494644165),
 ('jockey', 0.6354342699050903),
 ('breeders', 0.6333900690078735),
 ('race', 0.6270371675491333)]

## Подготовка данных для w2v

In [23]:
whole_sentence = df_train['Fixed_sentence1'] + ' : ' + df_train['Fixed_sentence2']

In [24]:
whole_bin_sentence = df_train['Fixed_bin_sentence1'] + ' : ' + df_train['Fixed_bin_sentence2']

In [25]:
neg_dif = whole_sentence.apply(isNeg)

In [26]:
words_counter = whole_sentence.apply(lambda sentence: countWords(sentence.split(' : ')[0]) / countWords(sentence.split(' : ')[1]))

In [27]:
def getSentenceVecSum(sentence):
  global model
  sentence = sentence.replace(')', ' ')
  sentence = sentence.replace('(', ' ')

  result_vec = np.zeros(100)
  for word in sentence.split():
      try:
          result_vec += model.wv[word]
      except:
          pass
    
  return result_vec

In [28]:
new_meaning_vecs = whole_bin_sentence.apply(lambda sentence: getMeaningVecs(sentence.split(' : ')[0], sentence.split(' : ')[1]))

<ipython-input-27-010c5506599d>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  result_vec += model.wv[word]


## Word2Vec + Logistic regression

In [29]:
def getDistanceBetweenVecs(vec1, vec2):
    return np.linalg.norm(vec1-vec2)

## Логистическая регрессия по смыслу слов

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


sentence1 = df_train.Fixed_sentence1.apply(lambda sentence: getSentenceVecSum(sentence))
sentence2 = df_train.Fixed_sentence2.apply(lambda sentence: getSentenceVecSum(sentence))

<ipython-input-27-010c5506599d>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  result_vec += model.wv[word]


In [31]:
x = sentence1 - sentence2

x

0         [0.2302660197019577, 0.660622013732791, 1.4518...
1         [1.0350759774446487, 2.03613199852407, 0.72746...
2         [1.4307060092687607, 0.8914919998496771, 0.777...
3         [0.517536997795105, -0.06919004023075104, -0.7...
4         [1.4938320070505142, -1.456748977303505, -0.36...
                                ...                        
549276    [-1.1741630211472511, -0.9560000151395798, -2....
549277    [-2.554003030061722, -2.694529950618744, -2.01...
549278    [-1.287889003753662, 1.396524053066969, 0.6614...
549279    [0.04770695045590401, 3.1734850481152534, -0.4...
549280    [0.4267420023679733, -0.24646998941898346, -1....
Length: 549281, dtype: object

## Подготовка даннных для глобальной модели

In [32]:
new_x = []
for i in range(len(x)):
    new_x.append([])
    for j in range(len(x[0])):
        new_x[i].append(x[i][j])

X = pd.DataFrame(new_x)

y = df_train.gold_label.apply(lambda label: {'entailment': 1, 'contradiction': -1, 'neutral': 0, '-': 3}[label])
print('>> Данные подготовлены для линейной регрессии')


>> Данные подготовлены для линейной регрессии


In [33]:
series1 = new_meaning_vecs.apply(lambda x: x[0])
series2 = new_meaning_vecs.apply(lambda x: x[1])
series3 = new_meaning_vecs.apply(lambda x: x[2])

In [34]:
new_1 = []
for i in range(len(series1)):
    new_1.append([])
    for j in range(len(series1[0])):
        new_1[i].append(series1[i][j])

new_1 = pd.DataFrame(new_1)

In [35]:
new_2 = []
for i in range(len(series2)):
    new_2.append([])
    for j in range(len(series2[0])):
        new_2[i].append(series2[i][j])

new_2 = pd.DataFrame(new_2)

In [36]:
new_3 = []
for i in range(len(series3)):
    new_3.append([])
    for j in range(len(series3[0])):
        new_3[i].append(series3[i][j])

new_3 = pd.DataFrame(new_3)

In [37]:
new_dict = {}

for j in range(100):
  new_dict[j] = 100+j
new_2 = new_2.rename(columns=new_dict)

for j in range(100):
  new_dict[j] = 200+j
new_3 = new_3.rename(columns=new_dict)

In [38]:
x = pd.concat([new_1, new_2, new_3], axis=1)

In [39]:
print('>> Данные подготовлены для psucho vecs регрессии')

>> Данные подготовлены для psucho vecs регрессии


In [40]:
X[100] = checker[0]


In [41]:
random_seed = 12

x_train, x_educ, y_train, y_educ = train_test_split(x, y, test_size=0.3, random_state=random_seed)
X_train, X_educ, y_train, y_educ = train_test_split(X, y, test_size=0.3, random_state=random_seed)


## Обучение логистической регрессии на psucho vecs

In [42]:
clf1 = LogisticRegression()
preds1 = clf1.fit(x_educ, y_educ).predict(x_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Обучение Гуасса для смыслового расстояния

In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [44]:
clf2 =  SVC(gamma='auto')
preds2 = clf2.fit(X_educ.head(5000), y_educ.head(5000)).predict(X_train)


## Построение финальной логистической регрессии

In [45]:
X_train[101] = preds1
X_train[102] = preds2


In [46]:
logRegr = LogisticRegression()
cv_results = cross_val_score(logRegr, X_train, y_train, cv=5, scoring='accuracy')

print('_'*80, f'>> Результаты кроссвалидации: {cv_results}', '_'*80, sep='\n')
print(cv_results.mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

________________________________________________________________________________
>> Результаты кроссвалидации: [0.55137841 0.55223085 0.55219184 0.55125554 0.55484467]
________________________________________________________________________________
0.5523802614379549


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Обучение финальной модели и прогнозирование на тестовых данных

In [47]:
with open(r'snli_1.0_test.jsonl', 'r') as json_file:
    json_list = list(json_file)

result = []

num = -1 # Сколько данных забрать (-1 чтобы забрать всё)

if num < 0:
    num = float('inf')

num = min(len(json_list), num)

for j in range(num):
    result.append(json.loads(json_list[j]))
    


In [48]:
df_test = pd.json_normalize(result)
df_test = df_test[['gold_label', 'sentence1', 'sentence2', 'sentence1_binary_parse', 'sentence2_binary_parse']]

In [49]:
checker2 = pd.DataFrame()

dis2 = np.loadtxt('D:\Monty_Python_lmao\Fumo parsing\distances_test.txt')
checker2[0]= dis2
checker2[checker2[0]>999] = 1.4



In [50]:
for i in [1, 2]:
    df_test[f'Fixed_sentence{i}'] = df_test[f'sentence{i}'].apply(lambda sentence: fixSentence(sentence, ['deleteSymbs']))

In [51]:
for i in [1, 2]:
    df_test[f'Fixed_bin_sentence{i}'] = df_test[f'sentence{i}_binary_parse'].apply(lambda sentence: fixSentence(sentence, ['deleteSymbs', 'delSpaces']))

In [52]:
whole_sentence_test = df_test['Fixed_sentence1'] + ' : ' + df_test['Fixed_sentence2']

In [53]:
whole_bin_sentence_test = df_test['Fixed_bin_sentence1'] + ' : ' + df_test['Fixed_bin_sentence2']

In [54]:
neg_dif_test = whole_sentence_test.apply(isNeg)

In [55]:
words_counter_test = whole_sentence_test.apply(lambda sentence: countWords(sentence.split(' : ')[0]) / countWords(sentence.split(' : ')[1]))

In [56]:
new_meaning_vecs_test = whole_bin_sentence_test.apply(lambda sentence: getMeaningVecs(sentence.split(' : ')[0], sentence.split(' : ')[1]))

<ipython-input-27-010c5506599d>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  result_vec += model.wv[word]


In [57]:
sentence1_test = df_test.Fixed_sentence1.apply(lambda sentence: getSentenceVecSum(sentence))
sentence2_test = df_test.Fixed_sentence2.apply(lambda sentence: getSentenceVecSum(sentence))

<ipython-input-27-010c5506599d>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  result_vec += model.wv[word]


In [58]:
x_test = sentence1_test - sentence2_test

x_test

0       [-1.7302690111100674, 5.868997022509575, 0.578...
1       [0.9498959593474865, 5.716138012707233, -0.464...
2       [-1.8306270353496075, 5.182091008871794, 0.808...
3       [-1.5425489461049438, 2.433895979076624, -2.07...
4       [-0.33247298654168844, 2.4271559603512287, -2....
                              ...                        
9995    [-1.201142966747284, 0.16192601714283228, -1.1...
9996    [-1.0620899721980095, 0.03271903097629547, -1....
9997    [0.1350870169699192, 2.1642130091786385, 1.134...
9998    [-0.7435459904372692, 1.8506419956684113, 0.67...
9999    [-1.7904659770429134, 0.6284220367670059, 1.76...
Length: 10000, dtype: object

In [59]:
new_x = []
for i in range(len(x_test)):
    new_x.append([])
    for j in range(len(x_test[0])):
        new_x[i].append(x_test[i][j])

X_test = pd.DataFrame(new_x)

y_test = df_test.gold_label.apply(lambda label: {'entailment': 1, 'contradiction': -1, 'neutral': 0, '-': 3}[label])
print('>> Данные подготовлены для линейной регрессии')

>> Данные подготовлены для линейной регрессии


In [60]:
series1_test = new_meaning_vecs_test.apply(lambda x: x[0])
series2_test = new_meaning_vecs_test.apply(lambda x: x[1])
series3_test = new_meaning_vecs_test.apply(lambda x: x[2])

In [61]:
new_1_test = []
for i in range(len(series1_test)):
    new_1_test.append([])
    for j in range(len(series1_test[0])):
        new_1_test[i].append(series1_test[i][j])

new_1_test = pd.DataFrame(new_1_test)

In [62]:
new_2_test = []
for i in range(len(series2_test)):
    new_2_test.append([])
    for j in range(len(series2_test[0])):
        new_2_test[i].append(series2_test[i][j])

new_2_test = pd.DataFrame(new_2_test)

In [63]:
new_3_test = []
for i in range(len(series3_test)):
    new_3_test.append([])
    for j in range(len(series3_test[0])):
        new_3_test[i].append(series3_test[i][j])

new_3_test = pd.DataFrame(new_3_test)

In [64]:
new_dict = {}

for j in range(100):
  new_dict[j] = 100+j
new_2_test = new_2_test.rename(columns=new_dict)

for j in range(100):
  new_dict[j] = 200+j
new_3_test = new_3_test.rename(columns=new_dict)

In [65]:
x_test = pd.concat([new_1_test, new_2_test, new_3_test], axis=1)

In [66]:
print('>> Данные подготовлены для psucho vecs регрессии')

>> Данные подготовлены для psucho vecs регрессии


In [67]:
X_test[100] = checker2[0]

In [68]:
preds1_test = clf1.fit(x_train, y_train).predict(x_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [69]:



preds2_test = clf2.fit(X_educ.head(15000), y_educ.head(15000)).predict(X_test)
# preds2_test = clf2.predict(X_test)



In [70]:
X_test[101] = preds1_test
X_test[102] = preds2_test


In [71]:
main_classifier = LogisticRegression()
main_classifier = main_classifier.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [72]:
test_pred = main_classifier.predict(X_test)

In [73]:
from sklearn.metrics import accuracy_score

In [74]:
accuracy_score(y_test, test_pred, normalize=True)

0.5624

In [75]:
from sklearn.metrics import confusion_matrix

In [76]:
con = confusion_matrix(y_test, test_pred)
con

array([[1687,  730,  820,    0],
       [ 823, 1606,  790,    0],
       [ 547,  490, 2331,    0],
       [  51,   51,   74,    0]], dtype=int64)

In [77]:
accuracy_score(y_test, test_pred, normalize=False)/ (np.sum(con)- np.sum(con[3]))

0.5724755700325733